In [47]:
import json

In [5]:
#import libraries that will be needed and input password, so viewers can't see it

import sqlalchemy
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup
import markdown
import getpass
import psycopg2
from openai import OpenAI  
import os 
engine = sqlalchemy.create_engine('postgresql://postgres@localhost:5432/myonet')

passwd = getpass.getpass("Password :")

Password :········


In [19]:
# Create 2 functions for working with the database
#
# getAllOccDataTitle - returns a result set of all the job names and job codes included in the ONet database
# addRecord - adds a record to the database in the ai table for each set of generated data for a job

def getJobData(jobcode):
    # iterate with first subscript, second subscript is always 0
    # x[341][0]
    try:
        connection = psycopg2.connect(user="postgres",
                                      password=passwd,
                                      host="127.0.0.1",
                                      port="5432",
                                      database="myonet")
        cursor = connection.cursor()
        sql_request_query = """ Select jobcode, jobname, industryjson, jobgroup, simoccsjson from jobdata_spanish where jobcode = %s"""
        cursor.execute(sql_request_query, (jobcode,))        
        z = cursor.fetchall()
    except (Exception) as error:
        print("Error in Update operation", error)
        return null
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")  
    return (z)

def getAllOccDataTitle():
    # iterate with first subscript, second subscript is always 0
    # x[341][0]
    try:
        connection = psycopg2.connect(user="postgres",
                                      password=passwd,
                                      host="127.0.0.1",
                                      port="5432",
                                      database="myonet")
        cursor = connection.cursor()
        sql_request_query = """ Select onetsoc_code, title, description from occupation_data"""
        cursor.execute(sql_request_query, ())        
        z = cursor.fetchall()
    except (Exception) as error:
        print("Error in Update operation", error)
        return null
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")  
    return (z)

def getWageData(onetsoc_code):
    try:
        connection = psycopg2.connect(user="altonhenley",
                                      password=passwd,
                                      host="127.0.0.1",
                                      port="5432",
                                      database="altonhenley")
        cursor = connection.cursor()
        sql_request_query = """ Select occ_code, occ_title, tot_emp, h_median, a_median, h_pct25, h_pct75, a_pct25, a_pct75, emp_prse
    from wagedata w
         where w.occ_code = %s
           and naics = '000000'
           and area_type = 1;"""
#         print(onetsoc_code[0:7])
        cursor.execute(sql_request_query, (onetsoc_code[0:7],))        
        z = cursor.fetchall()
#         print (z)
    except (Exception) as error:
        print("Error in Update operation", error)
        return null, null, null
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")  
    return z[0][0], z[0][1], z[0][2].replace(',',''), z[0][3], z[0][4].replace(',',''), z[0][5], z[0][6], z[0][7].replace(',',''), z[0][8].replace(',',''), z[0][9]


def addRecord(jobcode, jobname, longdescription, resume, pathway, steps):
    try:
        connection = psycopg2.connect(user="postgres",
                                      password=passwd,
                                      host="127.0.0.1",
                                      port="5432",
                                      database="myonet")    
        cursor = connection.cursor()
        sql_insert_query = """ INSERT INTO public.ai7(jobcode, jobname, longdescription, resume, pathway, steps) 
        VALUES (%s, %s, %s, %s, %s, %s); 
        """
        record_to_insert = (jobcode, jobname, longdescription, resume, pathway, steps)

        cursor.execute(sql_insert_query,record_to_insert)
        connection.commit()
        count = cursor.rowcount
#         print(count,"Record inserted successfully into table.")
    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into table", error)
    finally:
        if connection:
            cursor.close()
            connection.close()
            
def addRecordDescription(jobcode, jobname, longdescription):
    try:
        connection = psycopg2.connect(user="postgres",
                                      password=passwd,
                                      host="127.0.0.1",
                                      port="5432",
                                      database="myonet")    
        cursor = connection.cursor()
        sql_insert_query = """ INSERT INTO public.ai8(jobcode, jobname, longdescription) 
        VALUES (%s, %s, %s); 
        """
        record_to_insert = (jobcode, jobname, longdescription)

        cursor.execute(sql_insert_query,record_to_insert)
        connection.commit()
        count = cursor.rowcount
#         print(count,"Record inserted successfully into table.")
    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into table", error)
    finally:
        if connection:
            cursor.close()
            connection.close()

In [8]:
# Create functions for working with OpenAI API
#
# open_file - grabs data from a textfile (we use it to store the OpenAI API key to keep it secret)
# ask - the base routine for sending a prompt to OpenAI and receiving the result, its hardwired to use model gpt-4o
# askTerse - adds the position into a prompt asking for a short description and then passes that prompt to the ask function
# askLong - adds the position into a prompt asking for a long description and then passes that prompt to the ask function
# askPath - adds the position into a prompt asking for pathway and then passes that prompt to the ask function
# askResume - adds the position into a prompt asking for a resume and then passes that prompt to the ask function

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

def ask (prompt):
    key = open_file('openaiapikey.txt')
    client = OpenAI(
        # This is the default and can be omitted
        api_key=key,
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o",
    )
    text = chat_completion.choices[0].message.content
    return text

def askTerse(position):
    terseask = 'Write a short description of what a [[jobtitle]] does'
    terseask  = terseask.replace('[[jobtitle]]', position)
    result = ask(terseask)
    return result

def askLong(position):
    longask = 'Write an essay describing the job of a [[jobtitle]].'
    longask  = longask.replace('[[jobtitle]]', position)
    result = ask(longask)
    return result

def askPath (position):
    prompt =   "A career pathway is a list of the work experience, education, certifications and licenses, "  
    prompt +=  "skills development, and professional development necessary to be qualified for a position. " 
    prompt +=  "these terms should be used as section titles. What is the career pathway for being a [[position]]? " 
    prompt +=  "None of the elements should be numbered. the section titles should be displayed as a header. " 
    prompt +=  "the elements within each section should be displayed as a bullet point with a bold title followed "  
    prompt +=  "by a description format it using markdown"
    prompt = prompt.replace("[[position]]", position)
    pathtext = ask(prompt)
    return pathtext

def askResume(position):
    query = "Create a sample resume for someone looking for a job as [[position]] "
    resume = ask(query.replace("[[position]]", position))
    return resume

def askHiring(jobname):
    query = "Make a list of the most likely employers in [[place]] of a person with a career as a [[position]]. Include the name of the orgamization followed by tilde and then a link to the url of its job site followed by a tilde and a link to the logo of that organization. Just produce the list with no extra verbiage and format it using markdown."
    query = "Make a list of 10 of the major companies, from different industries if possible, that need to hire people as the following occupation, show the name of the company followed by tilde and then a link to the url of its job site, just produce the list with no extra verbiage and format it using markdown: [[position]] "
    hiring = ask(query.replace("[[position]]", jobname))
    return hiring

def justResume(text):
    pos = text.find("<hr />") + 8
    endpos = text.rfind("<hr />")
    return text[pos:endpos]

def askResume(position):
    query = "You are a career counselor for a workforce development board. Create a sample resume for someone looking for a job as [[position]] this resume should reflect suitable education, certifications, licenses and experience for someone eligible to land a job as a [[position]].  format it using markdown"

    resume = ask(query.replace("[[position]]", position))
    resume = markdown.markdown(resume)
    resume = doBold(resume).replace("\n", "\r\n")
    return justResume(resume)

def askLong2(position):
    longask = """
    You are a career navigator for a state college.
    Write a detailed job description for a [[jobtitle]]. Write it in the style of a college essay. 
    The description should include the following sections: 
    What They Do, Job Responsibilities, Essential Skills, Educational Pathways, Career Prospects,
    Conclusion. These terms should be used as section titles. It should be written in paragraph format.
    Format it using markdown. 
    """
    longask  = longask.replace('[[jobtitle]]', position)
    result = ask(longask)
    return result

def askLong2Spanish(position):
    longask = """
    You are a career navigator for a state college.
    Write a detailed job description for a [[jobtitle]]. Write it in the style of a college essay. 
    The description should include the following sections: 
    What They Do, Job Responsibilities, Essential Skills, Educational Pathways, Career Prospects,
    Conclusion. These terms should be used as section titles. It should be written in paragraph format.
    Format it using markdown. It should be written in spanish.
    """
    longask  = longask.replace('[[jobtitle]]', position)
    result = ask(longask)
    return result

def askDay(position):
    query = "write an essay covering a day in the work life of someone with the following occupation: [[position]] format it using markdown"
    day = ask(query.replace("[[position]]", position))
    return day

def askDaySpanish(position):
    query = "write an essay covering a day in the work life of someone with the following occupation: [[position]] . format it using markdown. the entire thing should be in spanish"
    day = ask(query.replace("[[position]]", position))
    return day

In [17]:
# the pathway prompt returns irregular results, this function normalizes the results

def cleanPathway(pathway):
    pathlist = []
    errlist = []
    flag = 0
    path = markdown.markdown(pathway)
    
    path = path.replace("</strong><ul>\n<li>", "</strong>")
    path = path.replace("</li>\n</ul>\n</li>", "</li>")
    path = path.replace("<h3>","<h1>").replace("</h3>","</h1>")
    path = path.replace("<h2>","<h1>").replace("</h2>","</h1>")
    path = path.replace("</strong></li>\n<li>", "</strong>")
    path = path.replace("</strong>\n</li>\n<li>", "</strong>")
    path = path.replace("<p>","")
    path = path.replace("</p>","")
    path = path.replace("</li>\n<li>\n<strong>","</li><li><strong>")
    path = path.replace("</strong>\n</li>\n<li>","</strong> ")
    path = path.replace("</h1>\n<ul>\n","</h1><ul>")
    path = path.replace("</h1>\n<ul>","</h1><ul>")
    path = path.replace("</li>\n<li>"," ")
    path = path.replace("<strong>","<li><strong>")
    path = path.replace("<li><li><strong>","<li><strong>")
    path = path.replace("</strong>",": </strong>")
    path = path.replace(":: </strong>",": </strong>")
    path = path.replace("\n","")
    path = path.replace("<ul><li><li>", "<ul><li>")
    path = path.replace("</strong>:", "</strong>")
    soup = BeautifulSoup(path) # If this line causes an error, run 'pip install html5lib' or install html5lib 
    unordereds = soup.findAll('ul')
    for u in unordereds:
        if 'education' in u.find_previous('h1').text.lower():
            category = 'education'
        elif 'work experience' in u.find_previous('h1').text.lower():
            category = 'experience'
        elif 'certification' in u.find_previous('h1').text.lower():
            category = 'certification'
        elif 'skills' in u.find_previous('h1').text.lower():
            category = 'skills'
        elif 'professional' in u.find_previous('h1').text.lower():
            category = 'professional'

        eds = u.findAll('li')
        for e in eds:
            try:
                content = e.text
                if content.find(":") > -1 :
                    contents = content.split(":")
                    item = contents[0]
                    description = contents[1]
                else:
                    if content.find("\n") > -1:
                        contents = content.split("\n")
                        item = contents[0]
                        description = contents[1]
                    else:
                        item = "none"
                        description = content
                        flag = 1
            except Exception as error:
                print(len(errlist),jobcode, "---", jobname)
                print('type',type(e))
                item = "none"
                description = e
                errrec = {
                    'data': e,
                    'error': error
                }
                errlist.append(errrec)
            rec = {
                'category': category,
                'item': item,
                'description': description
            }
            pathlist.append(rec)
    return pathlist


In [35]:
# Loop through all jobnames in database and create long description, short description, resume and pathways for each

joblist = []

jobs = getAllOccDataTitle()
for job in tqdm(jobs):
    jobcode = job[0]
    jobname = job[1]
    long_description = askLong2(jobname) 
    long_description = markdown.markdown(long_description).replace("h3","h4").replace("h2","h3").replace("h1","h2")
#     resume = askResume(jobname)
#     pathway = askPath(jobname)
#     steps=cleanPathway(pathway)

    values_to_add = {
        "jobcode": jobcode,
        "title": jobname, 
        "longdescription": long_description
#         "pathway": pathway,
#         "steps": steps, 
#         "resume": resume
    }       
    joblist.append(values_to_add)
#     addRecord(jobcode, jobname, long_description, resume, pathway, json.dumps(steps))

PostgreSQL connection is closed


100%|██████████| 1016/1016 [4:09:28<00:00, 14.73s/it] 


In [27]:
joblist2 = []
for job in joblist:
    job['longdescription'] = markdown.markdown(job['longdescription'])
    joblist2.append(job)
    
joblist2[45]

{'jobcode': '11-9121.02',
 'title': 'Water Resource Specialists',
 'longdescription': "<h1>Water Resource Specialists Job Description</h1>\n<h2>What They Do</h2>\n<p>Water Resource Specialists are pivotal in the management, assessment, and protection of water resources. They play a crucial role in ensuring that water bodies such as lakes, rivers, and reservoirs are kept clean, safe, and sustainable for human use and ecological health. Their work often involves analyzing water quality data, assessing environmental impacts, and developing conservation plans. These specialists work with public and private sector organizations, helping to inform policy, guide resource management, and provide technical expertise.</p>\n<h2>Job Responsibilities</h2>\n<p>Water Resource Specialists are tasked with a variety of responsibilities aimed at maintaining and improving water quality and availability. They conduct water quality monitoring, sampling, and testing to detect pollutants and assess compliance

In [36]:
joblist2[145]

{'jobcode': '15-2051.00',
 'title': 'Data Scientists',
 'longdescription': "<h1>Data Scientist Job Description</h1>\n<h2>What They Do</h2>\n<p>Data Scientists play a crucial role in the intersection of technology and data by extracting actionable insights from complex datasets. They utilize statistical, mathematical, and computational techniques to analyze large volumes of data, uncovering patterns and trends that can help organizations make informed decisions. These professionals often work across various industries including finance, healthcare, technology, and retail, offering a quantitative foundation for strategic planning, problem-solving, and predicting future trends.</p>\n<h2>Job Responsibilities</h2>\n<p>The responsibilities of a Data Scientist can be diverse and dynamic, reflecting the needs of the organization they work for. Common tasks include:</p>\n<ul>\n<li>Collecting and cleaning data from various internal and external sources.</li>\n<li>Applying statistical methods and

In [ ]:
# Option 1
# convert list of records into a datatable for export to excel
df = pd.DataFrame.from_records(joblist) 
df.to_excel('excelexportfile.xlsx')

In [38]:
# Option 2
# insert records directly into database
for item in tqdm(joblist):
    jobcode = item['jobcode']
    jobname = item['title']
    long_description = item["longdescription"]
#     short_description = item["shortdescription"]
#     resume = item["resume"]
#     pathway = item['pathway']
#     steps   = json.dumps(item['steps'])
#     addRecord(jobname, jobcode, long_description, short_description, resume, pathway, steps)
    addRecordDescription(jobname, jobcode, long_description)

100%|██████████| 1016/1016 [00:36<00:00, 27.85it/s]


In [11]:
def askLong2(position):
    longask = """
    You are a career navigator for a state college.
    Write a detailed job description for a [[jobtitle]]. Write it in the style of a college essay. 
    The description should include the following sections: 
    What They Do, Job Responsibilities, Essential Skills, Educational Pathways, Career Prospects,
    Conclusion. These terms should be used as section titles. It should be written in paragraph format.
    Format it using markdown. 
    """
    longask  = longask.replace('[[jobtitle]]', position)
    result = ask(longask)
    return result

def askLong2Spanish(position):
    longask = """
    You are a career navigator for a state college.
    Write a detailed job description for a [[jobtitle]]. Write it in the style of a college essay. 
    The description should include the following sections: 
    What They Do, Job Responsibilities, Essential Skills, Educational Pathways, Career Prospects,
    Conclusion. These terms should be used as section titles. It should be written in paragraph format.
    Format it using markdown. It should be written in spanish.
    """
    longask  = longask.replace('[[jobtitle]]', position)
    result = ask(longask)
    return result

In [33]:
b = askLong2('Data Scientists')
b

"# Exploring the Role of a Data Scientist\n\n## What They Do\n\nIn the contemporary digital age, data has become the lifeblood of decision-making and innovation. Data Scientists, at the forefront of this transformation, are highly skilled professionals adept at extracting meaningful insights from vast amounts of data. Primarily, they apply statistical analysis, machine learning, and specialized algorithms to interpret data patterns that inform business strategy, scientific research, and public policy. They are the bridge between raw, unstructured data and strategic decision-making, making their role crucial in numerous industries.\n\n## Job Responsibilities\n\nThe responsibilities of Data Scientists are diverse and multidimensional. They begin with defining the problem that needs solving and translating it into a data-oriented question. Subsequently, they collect, clean, and preprocess data to ensure its quality and relevance. Utilizing advanced statistical techniques and machine learn

In [34]:
b = markdown.markdown(b).replace("h3","h4").replace("h2","h3").replace("h1","h2")
b

"<h2>Exploring the Role of a Data Scientist</h2>\n<h3>What They Do</h3>\n<p>In the contemporary digital age, data has become the lifeblood of decision-making and innovation. Data Scientists, at the forefront of this transformation, are highly skilled professionals adept at extracting meaningful insights from vast amounts of data. Primarily, they apply statistical analysis, machine learning, and specialized algorithms to interpret data patterns that inform business strategy, scientific research, and public policy. They are the bridge between raw, unstructured data and strategic decision-making, making their role crucial in numerous industries.</p>\n<h3>Job Responsibilities</h3>\n<p>The responsibilities of Data Scientists are diverse and multidimensional. They begin with defining the problem that needs solving and translating it into a data-oriented question. Subsequently, they collect, clean, and preprocess data to ensure its quality and relevance. Utilizing advanced statistical techniq

In [49]:
jobcode = '00-0000.00'
jobcode_summ = jobcode[0:5] + '00'

In [50]:
getWageData(jobcode_summ)

Error in Update operation connection to server at "127.0.0.1", port 5432 failed: FATAL:  password authentication failed for user "altonhenley"



UnboundLocalError: cannot access local variable 'connection' where it is not associated with a value

In [9]:
eng = askLong2('Personal Chef')
markdown.markdown(eng)

"```markdown\n# Personal Chef: A Career Exploration\n\nIn today's fast-paced world, the demand for convenient, personalized dining experiences continues to grow. This trend has given rise to the profession of personal chefs, who cater to individual culinary preferences with an unmatched flair. This essay delves into the multifaceted world of personal chefs by exploring what they do, their job responsibilities, essential skills required, potential educational pathways, and the career prospects, concluding with a succinct overview of this intriguing profession.\n\n## What They Do\n\nA personal chef designs and prepares customized meals for individuals, families, or specific clientele. Unlike private chefs who work full-time for a family or household, personal chefs typically serve multiple clients and tailor their services to meet a diverse array of dietary requirements and taste preferences. Their primary role is to craft delicious and nutritious meals in the comfort of clients' homes o

In [12]:
spa = askLong2Spanish('Personal Chef')
spa

'# Descripción de Trabajo de un Chef Personal\n\n## Qué Hacen\n\nUn Chef Personal es un profesional culinario que se especializa en proporcionar experiencias gastronómicas personalizadas adaptadas a las necesidades y preferencias individuales de sus clientes. A diferencia de los chefs de restaurante, que cocinan para una clientela diversa, el Chef Personal se centra en crear menús únicos para individuos, familias, o grupos pequeños. Esto puede incluir la preparación de comidas diarias, la creación de menús para eventos especiales, o incluso ofrecer clases de cocina privadas. El objetivo principal es ofrecer una experiencia gastronómica que combine el arte culinario con un enfoque personalizado.\n\n## Responsabilidades del Trabajo\n\nLas responsabilidades de un Chef Personal son amplias y variadas, lo que requiere un enfoque meticuloso y creativo. En primer lugar, deben reunirse con los clientes para discutir sus necesidades dietéticas y preferencias culinarias. Basado en estas discusio

In [13]:
markdown.markdown(eng)

"<p>```markdown</p>\n<h1>Personal Chef: A Career Exploration</h1>\n<p>In today's fast-paced world, the demand for convenient, personalized dining experiences continues to grow. This trend has given rise to the profession of personal chefs, who cater to individual culinary preferences with an unmatched flair. This essay delves into the multifaceted world of personal chefs by exploring what they do, their job responsibilities, essential skills required, potential educational pathways, and the career prospects, concluding with a succinct overview of this intriguing profession.</p>\n<h2>What They Do</h2>\n<p>A personal chef designs and prepares customized meals for individuals, families, or specific clientele. Unlike private chefs who work full-time for a family or household, personal chefs typically serve multiple clients and tailor their services to meet a diverse array of dietary requirements and taste preferences. Their primary role is to craft delicious and nutritious meals in the com

In [14]:
markdown.markdown(spa)

'<h1>Descripción de Trabajo de un Chef Personal</h1>\n<h2>Qué Hacen</h2>\n<p>Un Chef Personal es un profesional culinario que se especializa en proporcionar experiencias gastronómicas personalizadas adaptadas a las necesidades y preferencias individuales de sus clientes. A diferencia de los chefs de restaurante, que cocinan para una clientela diversa, el Chef Personal se centra en crear menús únicos para individuos, familias, o grupos pequeños. Esto puede incluir la preparación de comidas diarias, la creación de menús para eventos especiales, o incluso ofrecer clases de cocina privadas. El objetivo principal es ofrecer una experiencia gastronómica que combine el arte culinario con un enfoque personalizado.</p>\n<h2>Responsabilidades del Trabajo</h2>\n<p>Las responsabilidades de un Chef Personal son amplias y variadas, lo que requiere un enfoque meticuloso y creativo. En primer lugar, deben reunirse con los clientes para discutir sus necesidades dietéticas y preferencias culinarias. Bas

In [23]:
z = getJobData('11-9041.00')
jobcode = z[0][0]
jobname = z[0][1]
industryjson = z[0][2]
jobgroup = z[0][3]
simoccsjson = z[0][4] 


PostgreSQL connection is closed


In [24]:
simoccsjson

'{ "13-1082.00" : "Project Management Specialists", "17-2112.00" : "Industrial Engineers", "17-2071.00" : "Electrical Engineers", "13-1081.01" : "Logistics Engineers", "17-2199.05" : "Mechatronics Engineers" }'

In [25]:
ask("translate the following into spanish: Project Management Specialists")

'Especialistas en gestión de proyectos'